In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2
from wordcloud import STOPWORDS
from sqlalchemy import create_engine 
import os
from dotenv import load_dotenv
load_dotenv("../.env.postgres")

True

In [2]:
conn = psycopg2.connect(
    database = "amazon_uk",
    user = os.environ["POSTGRES_USER"],
    password = os.environ["POSTGRES_PASSWORD"]
)

We first analyze the words used in title of top 5% most bought products vs. top 5% least bought products

In [3]:
df = pd.read_sql('SELECT * FROM amazon_uk_2023 ORDER BY "boughtInLastMonth" DESC LIMIT 100000;', conn)
df.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_6716\3787557939.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT * FROM amazon_uk_2023 ORDER BY "boughtInLastMonth" DESC LIMIT 100000;', conn)


,index,asin,title,imgUrl,productURL,stars,reviews,price,isBestSeller,boughtInLastMonth,categoryName
0,205639,B001MZV3OO,Catsan Hygiene Cat Litter 20L,https://m.media-amazon.com/images/I/71bURZaHfF...,https://www.amazon.co.uk/dp/B001MZV3OO,4.7,31204,14.99,False,50000,Pet Supplies
1,1557617,B004OCO20E,Andrex Gentle Clean Toilet Rolls - 45 Toilet R...,https://m.media-amazon.com/images/I/81K5VbVsCx...,https://www.amazon.co.uk/dp/B004OCO20E,4.6,44511,22.22,False,50000,Grocery
2,1252283,B07V2N4SJY,Amazon Brand – Mama Bear Sensitive Unscented B...,https://m.media-amazon.com/images/I/61lBVjA7jr...,https://www.amazon.co.uk/dp/B07V2N4SJY,4.7,32294,12.60,False,40000,Health & Personal Care
3,1557618,B01JS6YLQK,"Regina Blitz Household Towel, 560 Super-Sized ...",https://m.media-amazon.com/images/I/81+y0+Frsp...,https://www.amazon.co.uk/dp/B01JS6YLQK,4.8,40648,14.99,True,30000,Grocery
4,995347,B08WR5CPNB,Maybelline New York Lash Sensational Sky High ...,https://m.media-amazon.com/images/I/41vtg+FgTx...,https://www.amazon.co.uk/dp/B08WR5CPNB,4.4,52463,8.75,False,30000,Make-up


In [4]:
# check if a word has non alphabet character
def check_word(word):
    return all(w in "abcdefghijklmnopqrstuvwxyz" for w in word.lower())

In [5]:
stopwords = set(STOPWORDS)

In [6]:
most_bought_words = " ".join(title for title in df.title).lower().split(" ")
most_bought_words = [b for b in most_bought_words if (b not in stopwords) and (check_word(b))]
print(f"There are {len(most_bought_words)} words that in best seller category")

There are 1320546 words that in best seller category


In [7]:
most_bought_words_df = pd.DataFrame({"word": most_bought_words})
most_bought_words_df = pd.DataFrame(most_bought_words_df["word"].value_counts()).reset_index().rename({"index":"word", "word": "count"}, axis = 1)
most_bought_words_df.head()

,word,count
0,x,10983
1,pack,9640
2,hair,8472
3,kids,6876
4,set,6210


In [8]:
df = pd.read_sql('SELECT * FROM amazon_uk_2023 ORDER BY "boughtInLastMonth" LIMIT 100000;', conn)
least_bought_words = " ".join(title for title in df.title).lower().split(" ")
least_bought_words = [b for b in least_bought_words if (b not in stopwords) and (check_word(b))]
print(f"There are {len(least_bought_words)} words that in best seller category")

C:\Users\Admin\AppData\Local\Temp\ipykernel_6716\3764743998.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT * FROM amazon_uk_2023 ORDER BY "boughtInLastMonth" LIMIT 100000;', conn)


There are 1317531 words that in best seller category


In [9]:
most_bought_words_df = most_bought_words_df[~most_bought_words_df["word"].isin(set(least_bought_words))].reset_index().drop("index", axis = 1)
most_bought_words_df.head()

,word,count
0,nappy,215
1,gummies,187
2,keto,168
3,nicotine,167
4,whey,167


We also analyze the words used in title of top rated products (5 stars) (not in under 1 stars product)

In [10]:
df = pd.read_sql('SELECT * FROM amazon_uk_2023 WHERE stars = 5;', conn)
df.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_6716\2649471446.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT * FROM amazon_uk_2023 WHERE stars = 5;', conn)


,index,asin,title,imgUrl,productURL,stars,reviews,price,isBestSeller,boughtInLastMonth,categoryName
0,74,B09LYZM9RJ,"Fenton SHFS10W Active Subwoofer Speaker, 10"" f...",https://m.media-amazon.com/images/I/51jduec8al...,https://www.amazon.co.uk/dp/B09LYZM9RJ,5.0,1,169.00,False,0,Hi-Fi Speakers
1,267,B0BMB7B56J,Orange Portable Bluetooth Speaker Delivering S...,https://m.media-amazon.com/images/I/81R946t1Uf...,https://www.amazon.co.uk/dp/B0BMB7B56J,5.0,8,275.00,False,0,Hi-Fi Speakers
2,391,B0BHXJ72JT,Think Gizmos Portable Wireless Bluetooth Speak...,https://m.media-amazon.com/images/I/81sAggCZAJ...,https://www.amazon.co.uk/dp/B0BHXJ72JT,5.0,6,24.95,False,0,Hi-Fi Speakers
3,436,B0C8N8TMNM,"EnjoyNest Sound Bars for TV, 30 Watts Small So...",https://m.media-amazon.com/images/I/51jSjJI8nK...,https://www.amazon.co.uk/dp/B0C8N8TMNM,5.0,7,45.99,False,0,Hi-Fi Speakers
4,466,B0CDBV94S6,MAJORITY Bluetooth Sound Bar for TV with Wirel...,https://m.media-amazon.com/images/I/51UeNz2znO...,https://www.amazon.co.uk/dp/B0CDBV94S6,5.0,2,99.95,False,0,Hi-Fi Speakers


In [11]:
most_stars_words = " ".join(title for title in df.title).lower().split(" ")
most_stars_words = [b for b in most_stars_words if (b not in stopwords) and (check_word(b))]
print(f"There are {len(most_stars_words)} words that in best seller category")

There are 1251778 words that in best seller category


In [12]:
most_stars_words_df = pd.DataFrame({"word": most_stars_words})
most_stars_words_df = pd.DataFrame(most_stars_words_df["word"].value_counts()).reset_index().rename({"index":"word", "word": "count"}, axis = 1)
most_stars_words_df.head()

,word,count
0,gift,7733
1,x,7643
2,kids,6269
3,black,6251
4,set,5399


In [13]:
df = pd.read_sql('SELECT * FROM amazon_uk_2023 WHERE stars = 0;', conn)
least_stars_words = " ".join(title for title in df.title).lower().split(" ")
least_stars_words = [b for b in least_stars_words if (b not in stopwords) and (check_word(b))]
print(f"There are {len(least_stars_words)} words that in best seller category")

C:\Users\Admin\AppData\Local\Temp\ipykernel_6716\3928113250.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT * FROM amazon_uk_2023 WHERE stars = 0;', conn)


There are 16518357 words that in best seller category


In [14]:
most_stars_words_df = most_stars_words_df[~most_stars_words_df["word"].isin(set(least_stars_words))].reset_index().drop("index", axis = 1)
most_stars_words_df.head()

,word,count
0,nnme,30
1,winodws,30
2,homei,16
3,gigabti,15
4,tunderbolt,14


In [15]:
conn.close()

Save these datasets to Postgres

In [16]:
conn_string = os.environ.get("POSTGRES_CONN")
db = create_engine(conn_string) 
conn = db.connect() 

In [17]:
most_bought_words_df.to_sql('amazon_uk_2023_most_bought_words', conn, if_exists= 'replace') 

720

In [18]:
most_stars_words_df.to_sql('amazon_uk_2023_most_stars_words', conn, if_exists= 'replace') 

724

In [19]:
conn.close()